In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
load_dotenv("/home/kevin/projects/rag_langchain/.env")
def langsmith(project_name=None, set_enable=True):

    if set_enable:
        result = os.environ.get("LANGSMITH_API_KEY")
        if result is None or result.strip() == "":
            print(
                "LangChain API Key가 설정되지 않았습니다."
            )
            return
        os.environ["LANGCHAIN_ENDPOINT"] = (
            "https://api.smith.langchain.com"  # LangSmith API 엔드포인트
        )
        os.environ["LANGCHAIN_TRACING_V2"] = "true"  # true: 활성화
        os.environ["LANGCHAIN_PROJECT"] = project_name  # 프로젝트명
        print(f"LangSmith 추적을 시작합니다.\n[프로젝트명]\n{project_name}")
    else:
        os.environ["LANGCHAIN_TRACING_V2"] = "false"  # false: 비활성화
        print("LangSmith 추적을 하지 않습니다.")

langsmith(project_name="lcel_dev")

LangSmith 추적을 시작합니다.
[프로젝트명]
lcel_dev


# FewShotPromptTemplate

In [2]:
from langchain_openai import ChatOpenAI

In [3]:
llm = ChatOpenAI(model_name="gpt-4o",temperature=0)

In [4]:
question = "F1은 한 시즌의 몇 그랑프리를 진행하나요?"

In [5]:
llm.invoke(question).content

'포뮬러 1(F1) 시즌의 그랑프리 수는 매년 다를 수 있지만, 최근 몇 년 동안은 대략 20~23개의 그랑프리가 열리고 있습니다. 2023년 시즌에는 23개의 그랑프리가 예정되어 있습니다. 그랑프리 수는 FIA(국제자동차연맹)와 F1 조직위원회가 결정하며, 일정과 장소는 매년 변경될 수 있습니다.'

In [6]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

In [74]:
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

In [75]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:{answer}")

In [83]:
print(example_prompt.format(**examples[0]))

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인



In [84]:
prompt = FewShotPromptTemplate(
    # 예시들 예시 안에 {} 값(question, answer)에 대한 값을 넣어서 예시로 작성.
    examples=examples,
    # examples에서 키벨류로 작성한 (question,answer)를 받아줄 프롬프트 즉 예시에 적은 키를 {}값으로 줘야 한다.
    example_prompt=example_prompt,
    # 최종 프롬프트 템플릿 예시를 보고 작성해야 할 지시문을 적는다.
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"]
)

In [85]:
question = "아이폰을 처음 공개했을 때 스티브 잡스의 나이는 몇 살이야?"

In [86]:
final_prompt = prompt.format(question=question)

In [87]:
llm.invoke(final_prompt).content

'이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 아이폰이 처음 공개된 날짜는 언제인가요?\n중간 답변: 아이폰은 2007년 1월 9일에 처음 공개되었습니다.\n추가 질문: 스티브 잡스는 언제 태어났나요?\n중간 답변: 스티브 잡스는 1955년 2월 24일에 태어났습니다.\n최종 답변은: 스티브 잡스는 아이폰을 처음 공개했을 때 51세였습니다.'

In [88]:
chain = prompt | llm |StrOutputParser()

In [89]:
chain.invoke({"question":"스티브 잡스가 맥북에어 키노트를 발표했을 때 나이가 몇 살인가요?"})

'이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 스티브 잡스가 맥북에어 키노트를 발표한 해는 언제인가요?\n중간 답변: 스티브 잡스는 2008년 1월 15일에 맥북에어 키노트를 발표했습니다.\n추가 질문: 스티브 잡스는 언제 태어났나요?\n중간 답변: 스티브 잡스는 1955년 2월 24일에 태어났습니다.\n최종 답변은: 스티브 잡스는 맥북에어 키노트를 발표했을 때 52세였습니다.'

# Example Selector
예제가 많은 경우 프롬프트에 포함할 예제를 선택해야 할 수도 있습니다. Example Selector는 이 작업을 담당하는 클래스입니다.

# Redis-stack 설치 방법
- 기존의 예제에서는 ChromaDB를 사용했는데 회사와 기존에 써봤던 VectorStore는 Redis라 Redis-stack을 설치
- 로컬에 직접 설치보다 Docker로 설치하는 것을 선호
- docker run -d -p 6379:6379 -p 8001:8001 redis/redis-stack:latest
- docker run -d -p 6380:6379 -p 8002:8001 redis/redis-stack:latest 내가 원하는 포트로 지정가능
``` 
docker exec -it [redis-stack container id] /bin/bash
redis-cli
ping->pong
```
- 위의 명령어로 직접 redis 터미널로 접속도 가능
- localhost:8001 접속시 GUI RedisInsight로 접속 가능

In [31]:
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector, SemanticSimilarityExampleSelector
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores.redis import Redis

In [39]:
redis_connection = Redis(redis_url="redis://localhost:6379", index_name="rag_test", embedding=OpenAIEmbeddings())


In [40]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    redis_connection,
    k=1
)

In [47]:
selected_examples = example_selector.select_examples({"question":question})

In [48]:
selected_examples

[{'id': 'doc:67f5855ef6a8414ebdae7bfccbc62c64:29cda937a0df4ae394331c33aee3486d',
  'question': '스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?',
  'answer': '이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 스티브 잡스는 몇 살에 사망했나요?\n중간 답변: 스티브 잡스는 56세에 사망했습니다.\n추가 질문: 아인슈타인은 몇 살에 사망했나요?\n중간 답변: 아인슈타인은 76세에 사망했습니다.\n최종 답변은: 아인슈타인\n'}]

In [50]:
print(f"입력에 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')

입력에 가장 유사한 예시:
아이폰을 처음 공개했을 때 스티브 잡스의 나이는 몇 살이야?

question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인



In [51]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=['question']
)

In [52]:
chain = prompt | llm |StrOutputParser()

In [53]:
answer = chain.stream({"question":"스티브 잡스가 살아 있을 때 출시된 아이폰 중에서 가장 최근 모델이 뭐야?"})

In [54]:
for token in answer:
    print(token,end="",flush=True)

스티브 잡스가 살아 있을 때 출시된 아이폰 중에서 가장 최근 모델은 아이폰 4S입니다. 아이폰 4S는 2011년 10월 4일에 발표되었고, 스티브 잡스는 그 다음 날인 2011년 10월 5일에 사망했습니다.

## MaxMarginalSelector

In [55]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    redis_connection,
    k=1
)

In [56]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=['question']
)

In [57]:
chain = prompt | llm |StrOutputParser()

In [59]:
chain.invoke({'question':"스티브 잡스가 살아 있을 때 출시된 아이폰 중에서 가장 최근 모델이 뭐야?"})

'스티브 잡스가 살아 있을 때 출시된 아이폰 중에서 가장 최근 모델은 아이폰 4S입니다. 아이폰 4S는 2011년 10월 4일에 발표되었고, 스티브 잡스는 그 다음 날인 2011년 10월 5일에 사망했습니다.'

In [ ]:
# FewShotChatMessagePromptTemplate

In [63]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate,ChatPromptTemplate

In [61]:
examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
   - 회의는 김수진 팀장의 개회사로 시작됨.
   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

2. 시장 동향 개요 (김수진)
   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

3. 디지털 마케팅 전략 (박지민)
   - 박지민은 디지털 마케팅 전략에 대해 발표.
   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

4. 소셜 미디어 캠페인 (이준호)
   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

5. 종합 논의
   - 팀원들 간의 아이디어 공유 및 토론.
   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 박지민 담당.
""",
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]

In [62]:
examples

[{'instruction': '당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요',
  'input': '2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.',
  'answer': '\n회의록: XYZ 회사 마케팅 전략 회의\n일시: 2023년 12월 25일\n장소: XYZ 회사 회의실\n참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n1. 개회\n   - 회의는 김수진 팀장의 개회사로 시작됨.\n   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.\n\n2. 시장 동향 개요 (김수진)\n   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.\n   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.\n\n3. 디지털 마케팅 전략 (박지민)\n   - 박지민은 디지털 마케팅 전략에 대해 발표.\n   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.\n\n4. 소셜 미디어 캠페인 (이준호)\n   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.\n   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.\n\n5. 종합 논의\n   - 팀원들 간의 아이디어 공유 및 토론.\n   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.\n\n6. 마무리\n   - 다음 회의 날짜 및 시간 확정.\n   - 회의록 정리 및 배포는 박지민 담당.\n'},
 {'instruction': '당신은 요약 전문가

In [65]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human","{instruction}:\n{input}"),
        ("ai","{answer}")
    ]
)


In [66]:
example_selector =SemanticSimilarityExampleSelector.from_examples(
    #선택가능한 예시 목록
    examples,
    #임베딩할 클래스
    OpenAIEmbeddings(),
    #벡터 스토어
    redis_connection,
    #꺼내 올 예시 수
    k=1)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt
)

In [67]:
question={
    "instruction": "회의록을 작성해주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

In [68]:
example_selector.select_examples(question)

[{'id': 'doc:cbaf6db868114104b500eac5b6816129:a38ac85fe30e4e029c71966ce555bc23',
  'instruction': '당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요',
  'input': '2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.',
  'answer': '\n회의록: XYZ 회사 마케팅 전략 회의\n일시: 2023년 12월 25일\n장소: XYZ 회사 회의실\n참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n1. 개회\n   - 회의는 김수진 팀장의 개회사로 시작됨.\n   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.\n\n2. 시장 동향 개요 (김수진)\n   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.\n   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.\n\n3. 디지털 마케팅 전략 (박지민)\n   - 박지민은 디지털 마케팅 전략에 대해 발표.\n   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.\n\n4. 소셜 미디어 캠페인 (이준호)\n   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.\n   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.\n\n5. 종합 논의\n   - 팀원들 간의 아이디어 공유 및 토론.\n   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.\n\n6. 마무리\n  

In [70]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","너는 똑똑한 어시스턴트야"),
        few_shot_prompt,
        ("human","{instruction}\n{input}")
    ]
)

In [71]:
chain = final_prompt | llm |StrOutputParser()

In [73]:
answer = chain.stream(question)
for token in answer:
    print(token,end="",flush=True)

회의록: ABC 기술 회사 제품 개발 팀 주간 진행 상황 회의
일시: 2023년 12월 26일
장소: ABC 기술 회사 회의실
참석자: 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

1. 개회
   - 회의는 최현수 프로젝트 매니저의 개회사로 시작됨.
   - 회의의 목적은 새로운 모바일 애플리케이션 프로젝트의 현재 진행 상황을 검토하고 다가오는 마일스톤에 대한 계획을 수립하는 것.

2. 진행 상황 업데이트
   - 최현수: 프로젝트 전체 진행 상황 개요 제공.
   - 황지연: 개발 진행 상황 보고.
     - 주요 기능 구현 상태 및 현재까지의 문제점 공유.
     - 버그 수정 및 성능 최적화 작업 현황 설명.
   - 김태영: UI/UX 디자인 진행 상황 보고.
     - 사용자 인터페이스 디자인 완료 상태 및 피드백 반영 현황 설명.
     - 사용자 경험 개선을 위한 추가 작업 계획 공유.

3. 다가오는 마일스톤 계획
   - 최현수: 다음 마일스톤에 대한 주요 목표 및 일정 제시.
   - 팀원들 간의 협력 방안 논의.
   - 각 팀원의 역할과 책임 재확인.

4. 다음 주 목표 설정
   - 황지연: 다음 주까지 완료할 개발 작업 목록 제시.
   - 김태영: 다음 주까지 완료할 디자인 작업 목록 제시.
   - 최현수: 전체 프로젝트 일정 조정 및 리소스 배분 계획 설명.

5. 종합 논의
   - 팀원들 간의 추가 의견 교환 및 질의응답.
   - 예상되는 문제점 및 해결 방안 논의.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 최현수 담당.

회의 종료: 오후 4시 30분

작성자: 최현수 (프로젝트 매니저)

In [90]:
from langchain_teddynote.prompts import  CustomExampleSelector